In [ ]:
import sys
import os
import shutil
import tarfile
import pretty_midi
import fluidsynth
from IPython import display
import xml.etree.ElementTree as ET
import pprint as pp
import glob
import ntpath
import re
import json
import xml.dom.minidom
import numpy as np
from functools import total_ordering
import random
import json
import xmltodict

import midi_constants as mc

cwd = os.getcwd()

pattern_path = cwd + '/generated_patterns/'
pattern_midi_path = cwd + '/generated_pattern_midi_new/'

MAXCOUNT = 25
maxDim = 192

# Sampling rate for audio playback
_SAMPLING_RATE = 16000.0

In [ ]:
# delete all files and subdirectories in directory
def delete_directory(directory):
    for root, dirs, files in os.walk(directory, topdown=False):
        for name in files:
            os.remove(os.path.join(root, name))
        for name in dirs:
            os.rmdir(os.path.join(root, name))
    os.rmdir(directory)

# create new directory or delete directory and recreate if exists
def recreate_directory(directory):
    if  os.path.exists(directory):
        delete_directory(directory)
    os.makedirs(directory)

def display_audio(pm: pretty_midi.PrettyMIDI, seconds=30):
    waveform = pm.fluidsynth(fs=_SAMPLING_RATE)
    # Take a sample of the generated waveform to mitigate kernel resets
    waveform_short = waveform[:seconds*int(_SAMPLING_RATE)]
    return display.Audio(waveform_short, rate=int(_SAMPLING_RATE))

def get_midi_note_from_inst_number(inst:str):
    if inst.isnumeric():
        return int(inst) + 35 # might need refinement
    return None

def get_midi_time_from_pattern_position(position:str):
    if position.isnumeric():    
        return int(position)/15
    return None

In [ ]:
recreate_directory(directory=pattern_midi_path)

pattern_file_names = glob.glob(pattern_path + '*.h2pattern')
print('Number of files:', len(pattern_file_names))
print(pattern_file_names[0])



In [ ]:
def get_note_map(data_dict:dict):
        note_map = []
        for i in data_dict['drumkit_pattern']['pattern']['noteList']['note']:
                midi_note = {}
                try:
                        midi_note['instrument_number'] = get_midi_note_from_inst_number(i['instrument'])
                        midi_note['start_time'] = get_midi_time_from_pattern_position(i['position'])
                        note_map.append(midi_note)
                except Exception as e:
                        print(f'Caught exception {e}')
        return note_map

def generate_midi_file(note_map:list, midi_file_name:str):
        pm = pretty_midi.PrettyMIDI()
        for note in note_map:
                inst = note.get('instrument_number', 35)
                start_time = note.get('start_time')
                drum = pretty_midi.Instrument(program=inst, is_drum=True)
                note = pretty_midi.Note(
                        velocity=100, pitch=inst, start=start_time, end=start_time + .5)
                drum.notes.append(note)
                pm.instruments.append(drum)
        pm.write(midi_file_name)

In [ ]:

print(f'{len(pattern_file_names)=}')

secondary_pat_file_index = int(random.uniform(0, len(pattern_file_names)))
print(f'{secondary_pat_file_index}')


In [ ]:

counter = 0

for pat_file in pattern_file_names:
    counter = counter + 1
    # use for testing or limiting output
    if counter > MAXCOUNT:
        break
    data_dict = {}
    note_map_primary = []
    with open(pat_file) as fd:
        data_dict = xmltodict.parse(fd.read())
        note_map_primary = get_note_map(data_dict)
        
    # open secondary - use as option to create some unique midi files for further processing
    note_map_sec = []
    with open(pattern_file_names[secondary_pat_file_index]) as fd:
        data_dict = xmltodict.parse(fd.read())
        note_map_sec = get_note_map(data_dict)
        
    # note_map = note_map_primary + list(set(note_map_sec) - set(note_map_primary)) 
    note_map = note_map_primary + note_map_sec 
     
        
    m_file_name = mid_name = 'pattern_midi_' + str(counter).zfill(8) + '.midi'
    print(f'{m_file_name=}')    
    midi_file_name = os.path.join(pattern_midi_path, m_file_name)
    generate_midi_file(note_map, midi_file_name)

In [ ]:

# play it if you like
pm = pretty_midi.PrettyMIDI(pattern_midi_path + 'pattern_midi_00000001.midi')

display_audio(pm)
